In [1]:
pip install nltk


In [2]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [3]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [4]:
ssl._create_default_https_context=ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you", "What's up"],
        "responses": ["Hi there", "Hello", "Hey", "I'm fine, thank you", "Nothing much"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responses": ["Goodbye", "See you later", "Take care"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
        "tag": "travel",
        "patterns": [
            "Can you suggest a travel destination?",
            "Where should I go for vacation?",
            "Recommend me a place to visit"
        ],
        "responses": [
            "Sure! What type of destination do you prefer? Beaches, mountains, or cities?",
            "I recommend visiting Bali for a tropical experience, or Paris for a romantic getaway!",
            "You can explore Kyoto if you love cultural heritage and nature."
        ]
    },
    {
        "tag": "food",
        "patterns": [
            "What should I eat today?",
            "Can you suggest a good dish?",
            "Recommend me some food"
        ],
        "responses": [
            "How about trying a homemade pizza or a healthy salad?",
            "If you like spicy food, go for Indian curry. Otherwise, a classic burger never fails!",
            "Sushi is always a great choice for a light and delicious meal!"
        ]
    },
    {
        "tag": "time",
        "patterns": [
            "What time is it?",
            "Tell me the current time",
            "What's the date today?"
        ],
        "responses": [
            "Let me check... [Real-time Time Display Placeholder]",
            "Right now, it's [Real-time Time Display Placeholder].",
            "Today's date is [Real-time Date Placeholder]."
        ]
    },
    {
        "tag": "news",
        "patterns": [
            "What's the latest news?",
            "Tell me today's headlines",
            "Give me some updates"
        ],
        "responses": [
            "I can fetch real-time news updates for you! [News API Integration Placeholder]",
            "Let me check the latest news for you...",
            "Here are some trending stories: [Real-time News Placeholder]"
        ]
    },
    {
        "tag": "fitness",
        "patterns": [
            "How can I lose weight?",
            "Give me a workout routine",
            "Suggest a fitness plan"
        ],
        "responses": [
            "A balanced diet and regular exercise is key! Try combining cardio with strength training.",
            "For a beginner workout: Start with 10 push-ups, 15 squats, and a 30-second plank. Gradually increase the reps over time.",
            "Try intermittent fasting combined with a healthy meal plan. Also, stay hydrated!"
        ]
    },
    {
        "tag": "tech_news",
        "patterns": [
            "What's new in technology?",
            "Tell me about AI advancements",
            "Give me the latest tech trends"
        ],
        "responses": [
            "AI is evolving rapidly! Chatbots, self-driving cars, and automation are changing the world!",
            "Quantum computing is on the rise, and companies like Google and IBM are leading the way!",
            "Tech giants are investing in AI-driven healthcare, making diagnoses faster and more accurate!"
        ]
    }
]

In [6]:
#Create the vectorize and classifier
vectorizer = TfidfVectorizer()
clf=LogisticRegression(random_state=0,max_iter=1000)

#Preprocess the data
tags=[]
patterns=[]
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

In [7]:
#traing the model
x=vectorizer.fit_transform(patterns)
y=tags
clf.fit(x,y)

LogisticRegression(max_iter=1000, random_state=0)

In [14]:


def chatbot(input_text: str) -> str:
    if not input_text.strip():
        return "I didn't understand that. Can you please rephrase?"

    try:
        input_vector = vectorizer.transform([input_text])
        tag = clf.predict(input_vector)[0]

        # Convert intents list to a dictionary for faster lookup
        intent_dict = {intent['tag']: intent['responses'] for intent in intents}

        if tag in intent_dict:
            return random.choice(intent_dict[tag])
        else:
            return "I'm not sure how to respond to that."

    except Exception as e:
        return f"An error occurred: {str(e)}"


In [15]:
user_input="budget"
response=chatbot(user_input)
print(response)

An error occurred: index 1 is out of bounds for axis 0 with size 1


In [16]:
user_input="goodbye"
response=chatbot(user_input)
print(response)

An error occurred: index 1 is out of bounds for axis 0 with size 1


In [17]:
user_input="time"
response=chatbot(user_input)
print(response)

An error occurred: index 1 is out of bounds for axis 0 with size 1
